In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder("float", shape=[None, 784], name = 'x')  
y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target')

x_image = tf.reshape(x, [-1,28,28,1], name="x_image")

In [4]:
# W is [row, col, channel, feature]
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1), name="W_conv1") 
b_conv1 = tf.Variable(tf.zeros([32]), name="b_conv1")

h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1, name="h_conv1")
h_pool1 = tf.nn.max_pool( h_conv1 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool1")

W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1), name="W_conv2") 
b_conv2 = tf.Variable(tf.zeros([64]), name="b_conv2")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2, name="h_conv2")
h_pool2 = tf.nn.max_pool( h_conv2 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool2")

In [5]:
# Build a fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name="h_pool2_flat")
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1), name = 'W_fc1')
b_fc1 = tf.Variable(tf.zeros([1024]), name = 'b_fc1')
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="h_fc1")


keep_prob = tf.placeholder("float", name="keep_prob")
h_fc1 = tf.nn.dropout(h_fc1, keep_prob, name="h_fc1_drop")

W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1), name = 'W_fc2')
b_fc2 = tf.Variable(tf.zeros([10]), name = 'b_fc2')

In [7]:
y=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="y")
cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [8]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 

In [13]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.05))) 
sess.run(tf.initialize_all_variables())# initialize the variables

for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1], keep_prob: 0.5}) # placeholder's none length is replaced by i:i+100 indexes

    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1], keep_prob: 1})
        print "step %d, training accuracy: %.3f"%(i, train_accuracy)

print  "test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images[0:250], y_target: mnist.test.labels[0:250], keep_prob: 1})
sess.close()

step 0, training accuracy: 0.190
step 10, training accuracy: 0.400
step 20, training accuracy: 0.660
step 30, training accuracy: 0.740
step 40, training accuracy: 0.810
step 50, training accuracy: 0.780
step 60, training accuracy: 0.870
step 70, training accuracy: 0.890
step 80, training accuracy: 0.890
step 90, training accuracy: 0.910
step 100, training accuracy: 0.950
step 110, training accuracy: 0.910
step 120, training accuracy: 0.920
step 130, training accuracy: 0.910
step 140, training accuracy: 0.930
step 150, training accuracy: 0.940
step 160, training accuracy: 0.940
step 170, training accuracy: 0.930
step 180, training accuracy: 0.940
step 190, training accuracy: 0.960
step 200, training accuracy: 0.960
step 210, training accuracy: 0.940
step 220, training accuracy: 0.960
step 230, training accuracy: 0.950
step 240, training accuracy: 0.930
step 250, training accuracy: 0.960
step 260, training accuracy: 0.940
step 270, training accuracy: 0.960
step 280, training accuracy: 0.

KeyboardInterrupt: 